In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie
using LinearAlgebra, LinearSolve,Statistics

cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
function norm_cyylinder(u)
    return max( abs( u[1] ), sqrt( u[2]^2 + u[3]^2 ) )
end
function condition(u, t, integrator)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm_cyylinder(linsolve.u) - ϵ_box
end
affect!(integrator) = terminate!(integrator)
function get_norm_αs(u)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm_cyylinder(linsolve.u), linsolve
end

get_norm_αs (generic function with 1 method)

In [3]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305
t = 2000.0; tspan = [0.0, t];

In [4]:
# index point from homoclinic curve
index_hom = 1
I0 = I0_hom[index_hom]; U0 = u0_hom[index_hom]

p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [0.0, 0.0, 0.0]

ds = CoupledODEs(TM, u0, p)
fp, _, _ = fixedpoints(ds, box, jacob_TM_)

ϵ_box = 1e-5
Ju0 = jacob_TM_(fp[1], p, 0);
fp_ = fp[1]

eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors

v1 = real(eigen_vectors[:, 1])
v2 = real(eigen_vectors[:, 2])
v3 = imag(eigen_vectors[:, 3])

v1 = normalize(v1, 2)
v1t = transpose(v1)

v2 = normalize(v2, 2)
v2t = transpose(v2)

v3 = normalize(v3, 2)
v3t = transpose(v3)

A = transpose([v1t; v2t; v3t])
A = Matrix(A)

3×3 Matrix{Float64}:
  0.998728    0.999899     0.0
 -0.0489879  -0.0141814    0.984697
  0.0119121   0.000705852  0.174274

In [5]:
N = 51
nrange = range(1, N - 1, step = 1)
dots = zeros(length(nrange), 3)

Es = Float64[]; xs = Float64[]; ys = Float64[]
α1su0 =  Float64[]; α2su0 =  Float64[]; α3su0 =  Float64[]
α1s =  Float64[]; α2s =  Float64[]; α3s =  Float64[]
colors = []
check_events = []
time_events = []

for (i, n) in enumerate(nrange)
    α1 = 0
    α2 = cos(2 * pi * n / N) * ϵ_box
    α3 = sin(2 * pi * n / N) * ϵ_box
    αs = [α1, α2, α3]

    push!(α2su0, α2)
    push!(α3su0, α3)

    dot = fp_ + A * αs
    dots[i, :] = dot
end

In [6]:
index = 1
cb = ContinuousCallback(condition, nothing, affect!)
u0 = SA[dots[index, 1], dots[index, 2], dots[index, 3]]
prob = ODEProblem(TM, u0, tspan, p)
sol = solve(prob, alg = Vern9(), abstol = 1e-14, reltol = 1e-14, callback = cb)

retcode: Terminated
Interpolation: specialized 9th order lazy interpolation
t: 11-element Vector{Float64}:
 0.0
 0.03516697972492334
 0.057912089963057176
 0.08257494407133834
 0.10709790607528605
 0.13252460602649885
 0.15669982109738212
 0.17905275837599952
 0.2040385968834346
 0.21901872776640494
 0.21901872776640494
u: 11-element Vector{SVector{3, Float64}}:
 [8.34581837482542, 0.738495733991004, 0.4382987815206269]
 [8.345883187619544, 0.7384947135830516, 0.4382988093663714]
 [8.345921664505921, 0.7384939414032903, 0.438298796447142]
 [8.345954328788741, 0.7384931021302178, 0.43829875296073956]
 [8.345972573027051, 0.7384923653065166, 0.43829868123169774]
 [8.345972422953217, 0.738491812286151, 0.4382985828767921]
 [8.345952255310666, 0.738491573792596, 0.43829847581045184]
 [8.345916387860177, 0.7384916557117557, 0.4382983749596052]
 [8.345859196859527, 0.7384921177557834, 0.4382982728057269]
 [8.345818253683541, 0.7384925781491067, 0.4382982226121672]
 [8.345818253683541, 0.7384

In [7]:
for i in range(1, length(sol.u), step  =1)
    norm, linsolve = get_norm_αs(sol[i])
    push!(α2s, linsolve[2])
    push!(α3s, linsolve[3])
end

In [8]:
width, height = 600, 500
markersize = 5.0
color = :black
xlabel = L"α_2"; ylabel = L"α_3"
xlabelsize = 25; ylabelsize = 25

f = Figure(resolution = (width, height))
ax = Axis(f[1, 1], xlabel = xlabel, ylabel = ylabel, xlabelsize = xlabelsize, ylabelsize = ylabelsize)

scatter!(ax, α2s, α3s, markersize = markersize, color = color)

scatter!(ax, α2su0, α3su0, markersize = markersize, color = :blue)

scatter!(ax, 0.0, 0.0, markersize = markersize, color = :green)
display(GLMakie.Screen(), f)

GLMakie.Screen(...)

In [9]:
width, height = 600, 500
markersize = 5.0
color = :black
xlabel = L"x"; ylabel = L"y"; zlabel = L"E"
xlabelsize = 25; ylabelsize = 25; zlabelsize = 25

tstart, tend = 1, length(sol)
f = Figure(resolution = (width, height))
#ax = Axis3(f[1, 1], xlabel = xlabel, ylabel = ylabel, zlabel = zlabel, xlabelsize = xlabelsize, ylabelsize = ylabelsize, zlabelsize = zlabelsize)

ax = LScene(f[1, 1], show_axis = true)
scale!(ax.scene, 50, 50, 1)

lines!(sol[2, tstart:tend], sol[3, tstart:tend], sol[1, tstart:tend], color = :deeppink, linewidth = 1.0)
scatter!(ax, dots[:, 2], dots[:, 3], dots[:, 1],  markersize = markersize, color = color)

scatter!(ax, fp_[2], fp_[3], fp_[1],  markersize = markersize, color = :blue)
display(GLMakie.Screen(), f)

GLMakie.Screen(...)